In [49]:
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
import  numpy as np 

import os



def func1(image):
    cv2.imshow('a' , image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    number = []
    image2 = cv2.cvtColor(image.copy() , cv2.COLOR_BGR2RGB)
    boxes = pytesseract.image_to_data(image2)
    boxes = boxes.split('\n')
    boxes2 = []
    
    for b in boxes:
        b = b.split()
        boxes2.append(b)
        
    boxes2 = boxes2[1:]
    for x, b in enumerate(boxes2):
        print(b)
        if len(b) == 12 and x != 0 and x != (len(boxes2)-1)  and (len(b[11]) == 4) and b[11].isdigit():
                if ((len(boxes2[x-1]) == 12) and (len(boxes2[x-1][11]) == 4) and (boxes2[x-1][11].isdigit()) ) and ((len(boxes2[x+1]) == 12)and (len(boxes2[x+1][11]) == 4) and (boxes2[x+1][11].isdigit()) ):
                    b1 = boxes2[x-1]
                    b2 = boxes2[x]
                    b3 = boxes2[x+1]
                    if ((int(b1[7])-int(b2[7])) <= int(b1[7])*25/100) and ((int(b1[7])-int(b3[7])) <= int(b1[7])*25/100):
                        for i in range( x-1 , x+2):
                            b1 = boxes2[i]
                            x , y, w, h =  int(b1[6]) , int(b1[7]), int(b1[8]) , int(b1[9])
                            image = cv2.rectangle(image , (x,y) , (w+x , y+h) , (255,0,0) , 2)
                            if len(number) != 3 :
                                number.append(b[11])
                        if len(number) == 3:
                            break
                                
    if len(number)!=3 :
        return image2 , '0'
    numberr= ''.join(number)
    cv2.imshow('img' , image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return image,  int(numberr)

# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
  
# org 
org = (50, 50) 
  
# fontScale 
fontScale = 1
   
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2
   



counter = 0
def wrap_manually(img):
    # Using cv2.putText() method 
    image = cv2.putText(img.copy(), 'Click on Top-Left Corner of Req Image', org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)
    
    circles = np.zeros((4,2),np.int)
    def mousePoints(event,x,y,flags,params):
        global counter
        if event == cv2.EVENT_LBUTTONDOWN:
            circles[counter] = x,y
            counter = counter + 1
 
    while True:
        img2 = img.copy()
        global counter
        if counter == 4:
            width, height = 700 , 800
            pts1 = np.float32([circles[0],circles[1],circles[2],circles[3]])
            pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
            matrix = cv2.getPerspectiveTransform(pts1,pts2)
            imgOutput = cv2.warpPerspective(img,matrix,(width,height))
            cv2.imshow("Output Image ", imgOutput)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            counter = 0
            
            return imgOutput
            #cv2.imwrite('a.jpg', imgOutput)
            break
        if counter == 1:
        
            image = cv2.putText(img.copy(), 'Click on Top-Right most Corner of Req Image', org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)
        if counter == 2:
      
            image = cv2.putText(img.copy(), 'Click on Bottom-Left most Corner of Req Image', org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)
        if counter == 3:
       
            image = cv2.putText(img.copy(), 'Click on Bottam-Right most Corner of Req Image', org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)    
        for x in range (0,4):
            cv2.circle(image,(circles[x][0],circles[x][1]),3,(0,255,0),cv2.FILLED)
        
        cv2.imshow("Original Image ", image)
        cv2.setMouseCallback("Original Image ", mousePoints)
        cv2.waitKey(1)
        

        
        
def find_id_1(image):  
    image2 = image.copy()
    image2, a = func1(image2)
    if a != '0':
        return image2 , a  
    orb = cv2.ORB_create(50000)
    kpi , desi = orb.detectAndCompute(image, None)
    sample_images_path = os.listdir('1. Sample')
    sample_images_path = sample_images_path[1:]
    for sample_image in sample_images_path:
        img = cv2.imread( '1. Sample/'+ sample_image)
        kp, des = orb.detectAndCompute(img, None)
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)
        matches = bf.match(desi, des)

        matches.sort(key = lambda x: x.distance)
        good = matches[:int(len(matches)*(25/100))]
        srcPts = np.float32([kpi[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dstPts = np.float32([kp[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
 
        M, _ = cv2.findHomography(srcPts, dstPts, cv2.RANSAC, 5.0)
        h, w = img.shape[:2]
        imgScan = cv2.warpPerspective(image.copy(), M, (w, h))
        image2 = imgScan.copy()
        number = []
        image3 = image2.copy()
        image2 = cv2.cvtColor(image2 , cv2.COLOR_BGR2RGB)
        boxes = pytesseract.image_to_data(image2)
        boxes = boxes.split('\n')
        boxes2 = []
    
        for b in boxes:
            b = b.split()
 
            boxes2.append(b)
        
        boxes2 = boxes2[1:]
        for x, b in enumerate(boxes2):
            if len(b) == 12 and x != 0 and x != (len(boxes2)-1)  and (len(b[11]) == 4) and b[11].isdigit():
    
                if ((len(boxes2[x-1]) == 12) and (len(boxes2[x-1][11]) == 4) and (boxes2[x-1][11].isdigit()) ) and ((len(boxes2[x+1]) == 12)and (len(boxes2[x+1][11]) == 4) and (boxes2[x+1][11].isdigit()) ):
                    b1 = boxes2[x-1]
                    b2 = boxes2[x]
                    b3 = boxes2[x+1]
                    if ((int(b1[7])-int(b2[7])) <= int(b1[7])*25/100) and ((int(b1[7])-int(b3[7])) <= int(b1[7])*25/100):
                        for i in range( x-1 , x+2):
                    
                            b1 = boxes2[i]
                       
                            x , y, w, h =  int(b1[6]) , int(b1[7]), int(b1[8]) , int(b1[9])
                            image3 = cv2.rectangle(image3 , (x,y) , (w+x , y+h) , (255,0,0) , 2)
                            if len(number) != 3 :
                                number.append(b[11])
                        if len(number) == 3:
                            break
        if len(number) == 3:
            numberr= ''.join(number)
            cv2.imshow('img' , image3)
            #cv2.imwrite('savedImage.jpg', image) 
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            return image3 , int(numberr)
    img = wrap_manually(image)
    return (func1(img))



def find_id_2(img):
    img = wrap_manually(img)
    print(func1(img))
    

In [47]:
#TEST IMAGES 
path_test_images = '2. Test Images'
test_images = os.listdir(path_test_images)
test_images = test_images[1:]

for x, test_image_name in enumerate(test_images):
    test_image = cv2.imread(path_test_images + '/' + test_image_name   ) 
    img , idd = find_id_1(test_image)
    print(idd)
    cv2.imwrite(f'3. Test Images Result/{test_image_name}' , img)

111111111111
843084308430
294029402940
446344634463


In [ ]:
#TEST IMAGES 2
path_test_images_2 = '4. Test Images 2'
test_images_2 = os.listdir(path_test_images_2)
test_images_2 = test_images_2[1:]
for x, test_image_name in enumerate(test_images_2):
    test_image = cv2.imread(path_test_images_2 + '/' + test_image_name   ) 
    img , idd = find_id_1(test_image)
    print(idd)
    cv2.imwrite(f'5. Test Images 2  Result/{test_image_name}' , img)

['1', '1', '0', '0', '0', '0', '0', '0', '700', '800', '-1']
['2', '1', '1', '0', '0', '0', '213', '91', '417', '125', '-1']
['3', '1', '1', '1', '0', '0', '213', '91', '417', '125', '-1']
['4', '1', '1', '1', '1', '0', '242', '91', '388', '46', '-1']
['5', '1', '1', '1', '1', '1', '242', '103', '140', '34', '88', 'Government']
['5', '1', '1', '1', '1', '2', '390', '101', '22', '35', '96', 'of']
['5', '1', '1', '1', '1', '3', '420', '101', '53', '35', '92', 'India']
['5', '1', '1', '1', '1', '4', '570', '91', '60', '26', '19', 'ase']
['4', '1', '1', '1', '2', '0', '213', '171', '103', '45', '-1']
['5', '1', '1', '1', '2', '1', '213', '171', '103', '45', '11', 'i']
['2', '1', '2', '0', '0', '0', '213', '226', '265', '133', '-1']
['3', '1', '2', '1', '0', '0', '213', '226', '265', '133', '-1']
['4', '1', '2', '1', '1', '0', '214', '226', '135', '32', '-1']
['5', '1', '2', '1', '1', '1', '214', '226', '70', '32', '88', 'Sanjeev']
['5', '1', '2', '1', '1', '2', '292', '226', '57', '24', '9